In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from utils import import_igaba_pkl

nAP='40'
singles_used_for_combos = ['F102G', 'F102Y',
              'S99A', 'S99G', 'S99C',
              'F104Y', 'F104H',
              'K253I', 'K253Y',
              'L178R', 'L178S',
              'R168P']
igabasnfr2 = 'S99A F102Y F104Y L178S'
igabasnfr2n = 'S99A F104H R168P'
f102g = 'F102G'

x_plot = 'DF/F_norm'
y_plot = 'num pixels_norm'# 'SNR_norm'
data = import_igaba_pkl(nAP, 'responsivepx-segmented-csvs') # 'ilastik-segmented-csvs' or 'responsivepx-segmented-csvs'

data_highlights_singles = data.loc[[ind for ind in data.index if ind in singles_used_for_combos]]
# data_highlights_combos = data.loc[[ind for ind in data.index if ind in highlights_combos]]
f,ax = plt.subplots(figsize=[8,6])

p=sns.color_palette()


# plot single and combo mutations
ax=sns.scatterplot(data=data, 
                   x=x_plot, y=y_plot, 
                   palette=['lightgray', 'dimgray'],
                   hue='is_combo_int',
                   s=50)

# marker for F102G
ax = sns.scatterplot(data=data.loc[[f102g]],
               x=x_plot, y=y_plot, 
               marker='o', 
               color = 'black',
               s=180)

# markers for mutants used to make combos
ax = sns.scatterplot(data=data_highlights_singles,
               x=x_plot, y=y_plot, 
               marker='P',
               color = 'black',
               s=150)

# marker for igabasnfr2
ax = sns.scatterplot(data=data.loc[[igabasnfr2]],
               x=x_plot, y=y_plot, 
               marker='o', 
               color = 'red',
               s=70)

# marker for igabasnfr2n
ax = sns.scatterplot(data=data.loc[[igabasnfr2n]],
               x=x_plot, y=y_plot, 
               marker='o', 
               color = 'blue',
               s=70)


# marker for igabasnfr
plt.scatter(1,1,marker='o', color='black', s=150)

ax.get_legend().remove()
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

ax.set_xlabel(x_plot, fontsize=14)
ax.set_ylabel(y_plot, fontsize=14)

f.savefig('figs/screening_scatter_{}_vs_{}.pdf'.format(x_plot.replace('/','_'),y_plot.replace('/','_')))

In [ ]:
# stats
# data.sample(5)

singles_bool = data['is_combo_int'] == 1 # 1: combos, 0: singles
n_singles = (data['is_combo_int'] == 0).sum()
print('Num mutants: {}'.format(n_singles))

# df/f > 1, pVal < 0.001
print('mutants with df/f > 1, pVal < 0.001: {}'.format(((data['DF/F_norm'] > 1) & 
                                                                      (data['DF/F_pval 2'] < 0.001) & 
                                                                      singles_bool).sum()))

# n responsive px norm > 1
print('mutants with n responsive px norm > 1 and df/f > 1: {}'.format(((data['DF/F_norm'] > 1) & 
                                                                        (data['DF/F_pval 2'] < 0.001) & 
                                                                        (data['num pixels_norm'] > 1) & 
                                                                        singles_bool).sum()))


# df/f < 0, pVal < 0.001
data_neg = data[((data['DF/F_norm'] < 0) & (data['DF/F_pval 2'] < 0.001) & singles_bool)]
print('mutants with df/f < 0, pVal < 0.001: {}'.format(data_neg.shape[0]))
print('R168P mutation:')
display(data[['DF/F_norm', 'num pixels_norm']].loc['R168P'])
display(data[['DF/F_norm', 'num pixels_norm']].loc['F102G'])
# df/f < -1, pVal < 0.001
print('mutants with df/f < -1, pVal < 0.001: {}'.format(((data['DF/F_norm'] < -1) & 
                                                                      (data['DF/F_pval 2'] < 0.001) & 
                                                                      singles_bool).sum()))

#  mutation with max # responsive pixels
print('mutant with max # responsive pixels')
data.loc[data[singles_bool]['num pixels_norm'].idxmax()][['DF/F_norm', 'num pixels_norm', 'construct_ids']]

# mutant with max df/f
print('pos-going mutant with max df/f')
display(data.loc[data[singles_bool]['DF/F_norm'].idxmax()][['DF/F_norm', 'num pixels_norm', 'construct_ids']])

# num mutants with DF/F > F102G and # responsive pixels > iGABASnFR (i.e. >1)
dff_F102G = data[['DF/F_norm']].loc['F102G'].to_numpy()[0]
print('mutants with df/f > dff(F102G), # responsive pixels (norm.) > 1: {}'.format(((singles_bool) & 
                                                                                    (data['DF/F_norm'] > dff_F102G) & 
                                                                                    (data['num pixels_norm'] > 1)).sum()))

In [ ]:
'''
joint grid plot for scatterplot
'''
g = sns.JointGrid(data=data,
             x=x_plot, y=y_plot,
             palette=['lightgray', 'dimgray'],
                   hue='is_combo_int', 
                  ratio = 3,
                 space=0,
                 height=10)
g.plot_joint(sns.scatterplot,s=50)
g.plot_marginals(sns.histplot, kde=False)

# additional markers

# markers for mutants used to make combos
sns.scatterplot(ax = g.ax_joint, data=data_highlights_singles,
               x=x_plot, y=y_plot, 
               marker='P',
               color = 'black',
               s=150)

# marker for igabasnfr2
ax = sns.scatterplot(ax = g.ax_joint, data=data.loc[[igabasnfr2]],
               x=x_plot, y=y_plot, 
               marker='o', 
               color = 'red',
               s=70)

# marker for igabasnfr2n
ax = sns.scatterplot(ax = g.ax_joint, data=data.loc[[igabasnfr2n]],
               x=x_plot, y=y_plot, 
               marker='o', 
               color = 'blue',
               s=70)

g.savefig(r'C:\Users\labadmin\Dropbox (HHMI)\janelia\meetings\illustrations\hero\scatter.pdf')